I will code this task in PyTorch. I will follow a traditional Machine Learning / Deep Learning pipeline. 
- Data Preparation: Cleaning and preparing data for training, validation and testing. Including the DataLoader for further training.
- Model: Design and implementation of my Deep Learning model architecture. The DataLoader to work according to the format prepared in the Data Preparation. Lastly, a Trainer Class to perform the training of the model.
- Evaluation: Test the model with AUC score.
- Testing: Test on the 50th week.

# TO-DO: 
- 3th cell change


# Data preparation

In [17]:
import pandas as pd
import numpy as np
from tqdm import tqdm as tqdm
import os
import torch
import sys
from datetime import datetime
from configer import Configer
from torch.utils.data import DataLoader
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go

In [18]:
#load dataset
data = pd.read_csv(f"data/csv/train.csv").sort_values(['i','t'])
#to numpy
customers = data['i'].to_numpy(dtype = np.int32)
times = data['t'].to_numpy(dtype = np.int32)
prods = data['j'].to_numpy(dtype = np.int32)
prices = data['price'].to_numpy(dtype = np.float32)
advertised = data['advertised'].to_numpy(dtype = np.bool)
#number max customers
n_cust = data.i.max() + 1
n_prod = data.j.max() + 1
T = data.t.max() + 1
#paths for saving/loading
hist_path = 'data/csv/hist_data.npz'
coup_path = 'data/csv/coup_data.npz'
#skip_cells
skip = 1
data

,i,j,t,price,advertised
4818,0,39,3,0.961396,1
9420,0,6,6,0.479493,1
14082,0,30,9,1.961365,0
15652,0,6,10,0.799155,0
15653,0,18,10,1.961464,0
...,...,...,...,...,...
73342,1999,3,46,0.699985,0
73343,1999,5,46,1.750835,0
73344,1999,6,46,0.799155,0
74968,1999,6,47,0.799155,0


Create a matrix with the customer purchasing history (customer i purchased product j at time t) and save it with a gzip compression. It will be a matrix of dimensions [max_customers, weeks, max_products].

In [4]:
hist_data = np.zeros([n_cust, T, n_prod], dtype=np.bool) 
for i in tqdm(range(n_cust)):
    for t in range(T):
        for j in range(n_prod):
            hist_data[i, t, j] = np.sum(prods[(customers == i) & (times == t)] == j) > 0

np.savez(hist_path, data=hist_data)

100%|██████████| 2000/2000 [04:09<00:00,  8.01it/s]


Calculate the value of the discount recieved by coustomer i in time t for product j. Will have the same size as hist_data. Based on the original price & advertised price. (Formula: (orig_price - adv_price)/ orig_price)

In [5]:
#coupon assignment: size of discount received by
coup_data = np.zeros([n_cust, T, n_prod], dtype=np.float32)
#product prices
price_orig = np.zeros([n_prod,1])

for j in range(n_prod):
    price_orig[j] = np.median(prices[(prods == j) & (advertised == False)])

for i in tqdm(range(n_cust)):
        for t in range(T):
            for j in np.arange(n_prod)[hist_data[i, t] == True]:
                price = prices[(customers == i) & (times == t) & (prods == j)]
                coup_data[i, t, j] = (price_orig[j]-price)/price_orig[j]

np.savez(coup_path, data=coup_data)


100%|██████████| 2000/2000 [00:07<00:00, 276.77it/s]


Now we will prepare the files for training and validation. I will save files as .pt to improve the performance of the DataLoader and not overload the memory usage of the computer. Let's start with the training and validation set preparation.

In [19]:
hist_data = np.load(hist_path)['data']
coup_data = np.load(coup_path)['data']
window_size = 10
ovsize = 9
#time ids for treat the weeks that we want to train
time_ids = [np.arange(0, T)[i:i + window_size] for i in range(0, T, window_size - ovsize)]
#product purchasing frequencies
hist_data_inf = hist_data.mean(1)
#lists definition
coup_data_ = []
hist_data_ = []
hist_data_inf_ = []
hist_data2 = []
metas_i = []
metas_t = []
#load copuon assignment
coup_data = np.load(coup_path)['data']
#loop for fill the data (customers buys data (T & T+1) / coupon data (T & T+1) / purchasing frequencies / discounts
for x, t_id in enumerate(time_ids):   
    end_t = t_id[-1]
    if end_t == T - 1: continue
    for i in range(n_cust):     
        coup_data_.append(coup_data[i, end_t])
        hist_data_.append(hist_data[i, t_id].astype(np.int32))
        hist_data_inf_.append(hist_data_inf[i])
        hist_data2.append(hist_data[i, end_t+1].astype(np.int32))
        metas_i.append(i)
        metas_t.append(t_id)

#save it on a dictionary
data_train = {
    'coup_data_': np.stack(coup_data_), #coup data +1
    'hist_data' : np.stack(hist_data_),
    'hist_data_inf' : np.stack(hist_data_inf_),
    'hist_data2' : np.stack(hist_data2),#hist_data +1
    'metas_i' : np.stack(metas_i),
    'metas_t' : np.stack(metas_t)
}


In [20]:
def makepath(desired_path, isfile = False):
    '''
    if the path does not exist make it
    :param desired_path: can be path to a file or a folder name
    :return:
    '''
    import os
    if isfile:
        if not os.path.exists(os.path.dirname(desired_path)):os.makedirs(os.path.dirname(desired_path))
    else:
        if not os.path.exists(desired_path): os.makedirs(desired_path)
    return desired_path

In [21]:
n_data = len(hist_data2)
np.random.seed(100)
vald_ids = np.random.choice(n_data, int(n_data*0.1), replace=False)
train_ids = list(set(range(n_data)).difference(set(vald_ids)))


#save train data to .pkl
path_output = makepath('output/train/hist_data.pt', isfile=True)
for k, v in data_train.items():
    torch.save(torch.tensor(v[train_ids]), path_output.replace('hist_data.pt', '%s.pt'%k))

#save validation data to .pkl
path_output = makepath('output/validation/hist_data.pt', isfile=True)
for k, v in data_train.items():
    torch.save(torch.tensor(v[vald_ids]), path_output.replace('hist_data.pt', '%s.pt'%k))


Let's perpare the testing data. The testing will be applied on the 50th week. The test set specifies discount on product 24. But it's an special case because it has never been discounted before.

In [22]:
prediction_example = pd.read_csv("data/csv/prediction_example.csv").sort_values(['i','j'])
promotion_schedule = pd.read_csv("data/csv/promotion_schedule.csv").sort_values(['j'])

test_coup_data_ = []
test_hist_data_ = []
test_hist_data_inf_ = []
test_hist_data2 = []
test_metas_i = []

current_coupon = np.zeros(n_prod)

for j in range(n_prod):
    discount = promotion_schedule.query('j == %d' % j).discount.to_numpy()
    if len(discount) > 0: current_coupon[j] = discount

T_test = 49
t_ids = range(T_test-window_size, T_test)

for i in list(set(prediction_example.i)):
    test_coup_data_.append(current_coupon)
    test_hist_data_.append(hist_data[i, t_ids].astype(np.int32))
    test_hist_data_inf_.append(hist_data_inf[i])
    test_hist_data2.append(prediction_example.query("i==%d"%i).prediction.to_numpy())
    test_metas_i.append(i)

data_test = {
    'coup_data_': np.stack(test_coup_data_), #coup data +1
    'hist_data' : np.stack(test_hist_data_),
    'hist_data_inf' : np.stack(test_hist_data_inf_),
    'hist_data2' : np.stack(test_hist_data2),#hist_data +1
    'metas_i' : np.stack(test_metas_i)
}

path_output = makepath("output/test/hist_data.pt", isfile = True)
for k, v in data_test.items():
    torch.save(torch.tensor(v), path_output.replace('hist_data.pt', '%s.pt'%k))

# DataLoader
I will create my own DataLoader. The main reason is because I will load the .pt files/dictionaries that I already prepared in the last step. 

In [33]:
import os
import torch
from torch.utils.data import Dataset
import glob


class SO1_DL(Dataset):
    def __init__(self, dataset_dir):
        self.ds = {}
        for data_fname in glob.glob(os.path.join(dataset_dir, '*.pt')):
            k = os.path.basename(data_fname).replace('.pt','')
            self.ds[k] = torch.load(data_fname)

    def __len__(self):
        return len(self.ds['hist_data'])

    def __getitem__(self, idx):
        return self.fetch_data(idx)

    def fetch_data(self, idx):
        return {k: self.ds[k][idx].type(torch.float32) for k in self.ds.keys()}

# Modelling 
I will apply Deep Learning for this task. My model have a convolutional layer that will generate a feature map for customer purchases. Also I will apply an activation function to this layer. Hopefully, the filters of the convolution will extract patterns on the historic data of the customer and would be able to make relations between consumer-products buys. Also I add some linear bottlenecks to help share information. I use those because of my past experience using Mobilenetv2 which apply this technique (https://arxiv.org/pdf/1801.04381.pdf). Then, I will use a softmax layer to predict the probability of a customers buying a product in a concrete week (goal of the task).

The number of neurons/filters have been benchmarket to see which combination works better. The final network is the one that gave me less loss and performed better.

In [34]:
from torch import nn, optim

class SO1MODEL(nn.Module):

    def __init__(self, window_size=10, n_class=40, n_neurons=256, **kwargs):
        super(SO1MODEL, self).__init__()

        self.dropout = nn.Dropout(p=0.1, inplace=False)
        self.ll = nn.LeakyReLU(negative_slope=0.2)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)

        H = 20
        L = 1
        self.histdat_conv1 = nn.Conv1d(window_size, H, 1, stride=1)
        self.histdat_dense1 = nn.Linear(n_class*H, 6, bias=False)

        self.coupdat_dense1 = nn.Linear(n_class, L, bias=False)
        self.hisdatinf_dense1 = nn.Linear(n_class, L, bias=False)
        self.ztp1_dense1 = nn.Linear(n_class* (2*H+4), n_class, bias=False) 

    def forward(self, coup_data_, hist_data, hist_data_inf):

        bs, window_size, J = hist_data.shape
        #print(hist_data.shape)
        #convolucio
        bh_out = self.ll(self.histdat_conv1(hist_data)).transpose(1,2).contiguous()
        #bottleneck!
        #linear1 in: sortida de conv (64,800)
        bh_bar_out = self.ll(self.histdat_dense1(bh_out.view(bs, -1)))
        #linear1 out: (64, 6) | multiply: weights linear1 (800,6) · linear1 out T: (6,64) 
        bh_bar_out = self.ll(torch.matmul(self.histdat_dense1.weight.transpose(0,1), bh_bar_out.transpose(0,1)).transpose(0,1)).view(bs, J, -1)
        #multi out = 64,40,20
        
        #bottleneck!!
        #linear2: in coup_data (64,40)
        coup_out = self.ll(self.coupdat_dense1(coup_data_.view(bs, -1)))
        #linear2: (64,1) | multiply: weights linear2 (40,1) · linear2 out (1,64)
        coup_out = self.sigmoid(torch.matmul(self.coupdat_dense1.weight.transpose(0,1), coup_out.transpose(0,1))).transpose(0,1).view(bs, J, 1)
        #multi out = 64,40,1      
        hisdatinf_out = self.ll(self.hisdatinf_dense1(hist_data_inf.view(bs, -1)))
        hisdatinf_out = self.sigmoid(torch.matmul(self.hisdatinf_dense1.weight.transpose(0,1), hisdatinf_out.transpose(0,1))).transpose(0,1).view(bs, J, 1)
        
        Z = torch.cat([bh_out, bh_bar_out,coup_data_.view([bs,J,1]), coup_out,hist_data_inf.view(bs, J, 1), hisdatinf_out  ], dim=-1)
        #Z: (64,40,44)
        
        Ztp1 = self.ztp1_dense1(Z.view(bs,-1))
        #Ztp1: (64,40)
        pvalue = self.sigmoid(Ztp1)
        result = {'hist_data2': pvalue}

        return result

model = SO1MODEL()
model

SO1MODEL(
  (dropout): Dropout(p=0.1, inplace=False)
  (ll): LeakyReLU(negative_slope=0.2)
  (sigmoid): Sigmoid()
  (softmax): Softmax(dim=1)
  (histdat_conv1): Conv1d(10, 20, kernel_size=(1,), stride=(1,))
  (histdat_dense1): Linear(in_features=800, out_features=6, bias=False)
  (coupdat_dense1): Linear(in_features=40, out_features=1, bias=False)
  (hisdatinf_dense1): Linear(in_features=40, out_features=1, bias=False)
  (ztp1_dense1): Linear(in_features=1760, out_features=40, bias=False)
)

# Training part
This class have all the functions needed to perform the training part. I also use a configuration dictionary to select the parameters for the training. I use a binary cross-entropy loss (BCE), Adam optimizer and in the end I print the training/validation loss.

In [35]:


class SO1_Trainer:
    
    def __init__(self, work_dir, ps):
        
        #seed the RNG for all devices (CPU & GPU)
        torch.manual_seed(ps.seed)
        #time training control
        starttime = datetime.now().replace(microsecond=0)
        #create directory
        ps.work_dir = makepath(work_dir, isfile = False)
        #logger
        print("|{}| Start training SO1 Model".format(ps.exp_id))
        kwargs = {'num_workers': ps.n_workers}
        #load dataloaders
        #training set
        ds_train = SO1_DL(dataset_dir=os.path.join(ps.dataset_dir, 'train'))
        self.ds_train = DataLoader(ds_train, batch_size=ps.batch_size, shuffle=True, drop_last=True, **kwargs)
        #validation set
        ds_val = SO1_DL(dataset_dir=os.path.join(ps.dataset_dir, 'validation'))
        self.ds_val = DataLoader(ds_val, batch_size=ps.batch_size, shuffle=True, drop_last=True, **kwargs)
        #test set
        ds_test = SO1_DL(dataset_dir=os.path.join(ps.dataset_dir, 'test'))
        self.ds_test = DataLoader(ds_test, batch_size=ps.batch_size, shuffle=True, drop_last=False)
        #print dataset sizes
        print("Dataset train size: {} validation size {} test size {}".format(
                                         len(self.ds_train.dataset),
                                         len(self.ds_val.dataset) ,
                                         len(self.ds_test.dataset)))
        
        #define the model
        self.so1_model = SO1MODEL(window_size=ps.window_size, n_class=ps.n_class)
        varlist = [var[1] for var in self.so1_model.named_parameters()]
        #optimizer
        self.optimizer = optim.Adam(varlist, lr=ps.base_lr, weight_decay=ps.reg_coef)
        #loss and verboses
        self.best_loss_total = np.inf
        self.epochs_completed = 0
        self.ps = ps
        #binary cross entropy
        self.BCELoss = nn.BCELoss()
        
    def _get_model(self):
        return self.so1_model.module
    
    def train(self):
        self.so1_model.train()
        save_every_it = len(self.ds_train) / self.ps.log_every_epoch
        train_loss_dict = {}
        for it, dorig in enumerate(self.ds_train):
            
            dorig = {k:dorig[k] for k in dorig.keys()}

            self.optimizer.zero_grad()
            drec = self.so1_model(coup_data_=dorig['coup_data_'], hist_data=dorig['hist_data'], hist_data_inf=dorig['hist_data_inf'])

            loss_total, cur_loss_dict = self.compute_loss(dorig, drec)
            loss_total.backward()
            self.optimizer.step()

            train_loss_dict = {k: train_loss_dict.get(k, 0.0) + v.item() for k, v in cur_loss_dict.items()}
            cur_train_loss_dict = {k: v / (it + 1) for k, v in train_loss_dict.items()}
            train_msg = SO1_Trainer.creat_loss_message(cur_train_loss_dict, exp_id=self.ps.exp_id,
                                                              epoch_num=self.epochs_completed, it=it, mode='train')

           
        print(train_msg)
        train_loss_dict = {k: v / len(self.ds_train) for k, v in train_loss_dict.items()}
        return train_loss_dict
    
    def evaluate(self, split_name='validation'):
        self.so1_model.eval()
        eval_loss_dict = {}
        data = self.ds_val if split_name == 'validation' else self.ds_test
        with torch.no_grad():
            for dorig in data:
                dorig = {k: dorig[k] for k in dorig.keys()}
                drec = self.so1_model(coup_data_=dorig['coup_data_'], hist_data=dorig['hist_data'], hist_data_inf=dorig['hist_data_inf'])
                _, cur_loss_dict = self.compute_loss(dorig, drec)

                eval_loss_dict = {k: eval_loss_dict.get(k, 0.0) + v.item() for k, v in cur_loss_dict.items()}

        eval_loss_dict = {k: v / len(data) for k, v in eval_loss_dict.items()}
        return eval_loss_dict
    
    def compute_loss(self, dorig, drec):

        loss_dict = {
            'BCE': self.BCELoss(drec['hist_data2'], dorig['hist_data2']),
            }

        loss_dict['loss_total'] = torch.stack(list(loss_dict.values())).sum()

        return loss_dict['loss_total'], loss_dict
    
    
    def do_training(self, num_epochs=None, message=None):
        starttime = datetime.now().replace(microsecond=0)
        if num_epochs is None: num_epochs = self.ps.num_epochs
        
        prev_lr = np.inf
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min', patience=7)
        
        self.train_loss_list = []
        self.eval_loss_list = []
        
        for epoch_num in range(1, num_epochs + 1):
            train_loss_dict = self.train()
            eval_loss_dict = self.evaluate()
            
            self.train_loss_list.append(train_loss_dict['loss_total'])
            self.eval_loss_list.append(eval_loss_dict['loss_total'])
            
            scheduler.step(eval_loss_dict['loss_total'])
            cur_lr = self.optimizer.param_groups[0]['lr']
            
            if cur_lr != prev_lr:
                print('--- Optimizer learning rate changed from %.2e to %.2e ---' % (prev_lr, cur_lr))
                prev_lr = cur_lr
            self.epochs_completed += 1
    
            with torch.no_grad():
                eval_msg = SO1_Trainer.creat_loss_message(eval_loss_dict, exp_id=self.ps.exp_id,
                                                              epoch_num=epoch_num-1, it=len(self.ds_val),
                                                               mode='valid')
                
                if eval_loss_dict['loss_total'] < self.best_loss_total:
                    self.ps.best_model_fname = makepath(os.path.join(self.ps.work_dir, 'snapshots', 'SO1_E%03d.pt' % (
                            self.epochs_completed)), isfile=True)
                    print(eval_msg + ' !! ')
                    self.best_loss_total = eval_loss_dict['loss_total']
                    torch.save(self.so1_model.module.state_dict() if isinstance(self.so1_model,
                                                                                     torch.nn.DataParallel) else self.so1_model.state_dict(),
                               self.ps.best_model_fname)

                else:
                    print(eval_msg)
           
        endtime = datetime.now().replace(microsecond=0)
        np.savez('experiments/experiment1/train_losses.npz', data=self.train_loss_list)
        np.savez('experiments/experiment1/val_losses.npz', data=self.eval_loss_list)
        print('Training done in %s! Best val total loss achieved: %.2e\n' % (endtime - starttime, self.best_loss_total))
        
    def show_train_val_loss(self, train_loss_list, eval_loss_list):
        #X and Y axis inputs for Plotly graph.

        plot_data = [
              go.Scatter(
              x=np.arange(len(train_loss_list)),
              y=train_loss_list,
              name= 'train_loss'
          ),
           go.Scatter(
              x=np.arange(len(eval_loss_list)),
              y=eval_loss_list,
              name='val_loss'
              )
          ]
        plot_layout = go.Layout(
              title='Train-Val loss',
                xaxis=go.layout.XAxis(
                    title=go.layout.xaxis.Title(
                    text='Epochs')
                ),
                yaxis=go.layout.YAxis(
                title=go.layout.yaxis.Title(
                text='Loss')
                    )
                )
        fig = go.Figure(data=plot_data, layout=plot_layout)
        pyoff.iplot(fig)
    

    
    @staticmethod
    def creat_loss_message(loss_dict, exp_id='XX', epoch_num=0, it=0,  mode='valid'):
        ext_msg = ' | '.join(['%s = %.2e' % (k, v) for k, v in loss_dict.items() if k != 'loss_total'])
        return '[%s] Epoch %03d - %s: [%s]' % (
                exp_id, epoch_num, mode, ext_msg)        


In [36]:
exp_id = 'experiment1'
work_dir = os.path.join("experiments", exp_id)

params = {
        'seed': 2,
        'window_size': 10,
        'n_class': 40,
        'n_neurons': 256,
        'batch_size': 64, 
        'n_workers': 10,
        'cuda_id': 0,
        'use_multigpu':False,
        'reg_coef': 5e-4,
        'base_lr': 5e-3,
        'best_model_fname': None,
        'log_every_epoch': 2,
        'exp_id': exp_id,
        'work_dir': work_dir,
        'num_epochs': 100,
        'dataset_dir': 'output',
    }

#training definition
supercap_trainer = SO1_Trainer(work_dir, ps=Configer(default_ps_fname=params, **params))
#parameters
ps = supercap_trainer.ps
#training part
print("Batch size: {} | Epochs: {}".format(ps.batch_size, ps.num_epochs))
#supercap_trainer.do_training()

|experiment1| Start training SO1 Model
Dataset train size: 70200 validation size 7800 test size 3
Batch size: 64 | Epochs: 100


In [37]:
#print losses
train_losses = np.load(os.path.join('experiments',exp_id,'train_losses.npz'))['data']
eval_losses = np.load(os.path.join('experiments',exp_id,'val_losses.npz'))['data']
supercap_trainer.show_train_val_loss(train_losses, eval_losses)

# Evaluation
Evaluation of the algorithm using AUC metric as states the assignment. Note that I evaluate with the epoch that had lower loss (SO1_E042). 

In [48]:
c2c = lambda tensor: tensor.detach().cpu().numpy()
from sklearn.metrics import roc_curve
from sklearn.metrics import auc as compute_auc
import json

def evaluate_error(dataset_dir, so1_model, so1_ps, splitname, batch_size=1):
    so1_model.eval()
    n_class = 40
    ds_name = dataset_dir
    BCELoss = torch.nn.BCELoss()

    ds = SO1_DL(dataset_dir=os.path.join(dataset_dir, splitname))
    print('%s dataset size: %s'%(splitname,len(ds)))
    ds = DataLoader(ds, batch_size=batch_size, shuffle=False, drop_last=False)

    all_auc = {}

    loss_mean = []
    with torch.no_grad():
        for dorig in ds:
            dorig = {k: dorig[k] for k in dorig.keys()}

            drec = so1_model(coup_data_=dorig['coup_data_'], hist_data=dorig['hist_data'], hist_data_inf=dorig['hist_data_inf'])
            loss_mean.append(BCELoss(drec['hist_data2'], dorig['hist_data2']))
            
            y_test = c2c(dorig['hist_data2'])
            y_score = c2c(drec['hist_data2'])
  
            if splitname == 'test':
                y_test = np.int32(y_test > 0.5)
            # Compute ROC curve and ROC area for each class
            for j in range(n_class):
                fpr, tpr, _ = roc_curve(y_test[:, j], y_score[:, j])
                auc = compute_auc(fpr, tpr)
                c = all_auc.get(j, []); c.append(auc.mean()); all_auc[j] = c.copy()

    valid_ids = {k: ~np.isnan(v) for k,v in all_auc.items()}
    final_results = {
        'BCE': float(c2c(torch.stack(loss_mean).mean())),
        'auc': {k: np.mean(np.stack(v)[valid_ids[k]]) for k, v in all_auc.items()},
    }

    outpath = makepath(os.path.join(so1_ps.work_dir, 'evaluations', 'ds_%s'%ds_name, os.path.basename(so1_ps.best_model_fname).replace('.pt','_%s.json'%splitname)),isfile=True)
    with open(outpath, 'w') as f:
        json.dump(final_results,f)

    return final_results


In [49]:
from configer import Configer


def expid2model(expr_dir):
    from configer import Configer

    if not os.path.exists(expr_dir): raise ValueError('Could not find the experiment directory: %s' % expr_dir)

    best_model_fname = sorted(glob.glob(os.path.join(expr_dir, 'snapshots', '*.pt')), key=os.path.getmtime)[-1]

    print(('Found SO1 Trained Model: %s' % best_model_fname))
    
    default_ps_fname = glob.glob(os.path.join(expr_dir,'*.ini'))[0]
    ps = Configer(default_ps_fname=default_ps_fname, work_dir = expr_dir, best_model_fname=best_model_fname)

    return ps, best_model_fname

def load_so1_model(expr_dir, so1_pt):

    import importlib
    import torch

    ps, trained_model_fname = expid2model(expr_dir)
    so1_pt.load_state_dict(torch.load(trained_model_fname, map_location='cpu'))
    so1_pt.eval()
    
    return so1_pt, ps


In [50]:

expr_code = 'experiment1'

trained_model_fname = 'SO1_E042.pt'
expr_dir = os.path.join('experiments', expr_code, "snapshots", trained_model_fname)

so1_model = SO1MODEL(window_size=ps.window_size, n_class=ps.n_class)
so1_model.load_state_dict(torch.load(expr_dir, map_location='cpu'))

so1_ps = supercap_trainer.ps
so1_ps.best_model_fname = expr_dir
so1_ps.dataset_dir = "output"


dataset_dir = so1_ps.dataset_dir


print('Model Found: [%s] [Running on dataset: %s] '%(so1_ps.best_model_fname, dataset_dir))
for splitname in ['train', 'validation']:
    print('------- %s ----------'%splitname.upper())
    results = evaluate_error(dataset_dir, so1_model, so1_ps, splitname, batch_size=512)
    print('BCE = %.2e' % (results['BCE']))
    print('AUC: %s' % (', '.join(['%s: %.2f'%(k,v) for k, v in results['auc'].items()])))

Model Found: [experiments/experiment1/snapshots/SO1_E042.pt] [Running on dataset: output] 
------- TRAIN ----------
train dataset size: 70200


/Users/lupotto/anaconda3/lib/python3.6/site-packages/sklearn/metrics/ranking.py:571: UndefinedMetricWarning:

No positive samples in y_true, true positive value should be meaningless



BCE = 8.37e-02
AUC: 0: 0.55, 1: 0.62, 2: 0.59, 3: 0.59, 4: 0.54, 5: 0.69, 6: 0.64, 7: 0.60, 8: 0.53, 9: 0.53, 10: 0.53, 11: 0.53, 12: 0.51, 13: 0.53, 14: 0.53, 15: 0.63, 16: 0.56, 17: 0.53, 18: 0.49, 19: 0.47, 20: 0.50, 21: 0.54, 22: 0.54, 23: 0.52, 24: 0.56, 25: 0.54, 26: 0.53, 27: 0.55, 28: 0.52, 29: 0.49, 30: 0.70, 31: 0.47, 32: 0.51, 33: 0.68, 34: 0.61, 35: 0.53, 36: 0.55, 37: 0.50, 38: 0.49, 39: 0.72
------- VALIDATION ----------
validation dataset size: 7800
BCE = 8.37e-02
AUC: 0: 0.61, 1: 0.63, 2: 0.60, 3: 0.60, 4: 0.41, 5: 0.68, 6: 0.65, 7: 0.54, 8: 0.53, 9: 0.47, 10: 0.56, 11: 0.52, 12: 0.55, 13: 0.57, 14: 0.54, 15: 0.64, 16: 0.58, 17: 0.57, 18: 0.56, 19: 0.39, 20: 0.31, 21: 0.56, 22: 0.48, 23: 0.68, 24: 0.57, 25: 0.47, 26: 0.50, 27: 0.44, 28: 0.53, 29: 0.47, 30: 0.70, 31: 0.53, 32: 0.48, 33: 0.68, 34: 0.64, 35: 0.48, 36: 0.50, 37: 0.51, 38: 0.42, 39: 0.71


# Test on week 50th
First, I create a ground truth csv consumers and products. Then I load the promotion schedule. And finally I make predictions on the 50th week. The last step is generate the predictions_results.csv

In [53]:
#create csv for predictions
#id_consumer 0..1999
#id_product 0..39

import csv

with open('data/csv/prediction_ground_truth.csv','w') as f1:
    writer=csv.writer(f1, delimiter=',',lineterminator='\n',)
    writer.writerow(["i","j","prediction"])
    for i in range(0,2000):
        for j in range(0,40):
            row = [i,j]
            writer.writerow(row)


In [54]:
# Test will be applied on week 50th
prediction_example = pd.read_csv("data/csv/prediction_ground_truth.csv")  
promotion_schedule = pd.read_csv("data/csv/promotion_schedule.csv")  
prediction_results = pd.DataFrame(columns=list(prediction_example.columns))

current_coupon = np.zeros(n_prod)
for j in range(n_prod):
    discount = promotion_schedule.query('j == %d' % j).discount.to_numpy()
    if len(discount) > 0: current_coupon[j] = discount

    
T_test = 49
tIds = range(T_test - window_size, T_test)

for d in prediction_example.iterrows():
    i, j = int(d[1].i), int(d[1].j)
    
    
    prediction = c2c(so1_model(
        coup_data_=torch.tensor(current_coupon[np.newaxis], dtype=torch.float32),
        hist_data=torch.tensor(hist_data[i, tIds][np.newaxis], dtype=torch.float32),
        hist_data_inf=torch.tensor(hist_data_inf[i][np.newaxis], dtype=torch.float32))['hist_data2'])[0, j]
        
    prediction_results = prediction_results.append({'i': int(i), 'j': int(j), 'prediction': prediction}, ignore_index=True)

prediction_results.to_csv("data/csv/prediction_results.csv", index=False)
